In [1]:
# Magics



In [2]:
# Imports

In [3]:
import argparse
import logging
import sys
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam
from pyro.util import ignore_jit_warnings

In [4]:
# Settings

In [5]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
seed = 0
batch_size = 8
learning_rate = 0.05
num_steps = 200

In [6]:
# Code

In [7]:
# Dataset loading
def load_sequences():
    
    transition = torch.Tensor([[0.9,  0.05, 0.05],
                              [0.05, 0.9, 0.05],
                              [0.05, 0.05, 0.9],
                              ]
                               )
    emission = torch.Tensor(
    [
        [0.6, 0.3, 0.1],
        [0.1, 0.7, 0.2],
        [0.2, 0.3, 0.5],
    ])
    
    samples = []
    
    for j in torch.range(0, 50):
        observations = []
        
        state = pyro.sample("x_{}_0".format(j),
                                    dist.Categorical(torch.Tensor([1.0/3, 1.0/3, 1.0/3])),
                                   )
        emission_p_t = emission[state]

        observation = pyro.sample("y_{}_0".format(j),
                                    dist.Categorical(emission_p_t),
                                   )
        for k in torch.range(1, 50):
            transition_p_t = transition[state]
            
            state = pyro.sample("x_{}_{}".format(j, k),
                                    dist.Categorical(transition_p_t),
                                   )
            emission_p_t = emission[state]
            
            observation = pyro.sample("y_{}_{}".format(j, k),
                                    dist.Categorical(emission_p_t),
                                   )
        
            observations.append(observation)
            
        sample = torch.Tensor(observations)
        samples.append(sample)

    sequences_tensor = torch.vstack(samples)
    
    return sequences_tensor, torch.Tensor(np.array([len(sequence) for sequence in sequences_tensor])).to(torch.int), emission, transition

In [8]:
sequences, lengths, emission, transition = load_sequences()



C:\Users\cwild\.conda\envs\nic\lib\site-packages\ipykernel_launcher.py:18: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
C:\Users\cwild\.conda\envs\nic\lib\site-packages\ipykernel_launcher.py:29: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).


In [9]:
sequences

tensor([[0., 2., 1.,  ..., 0., 1., 0.],
        [0., 1., 2.,  ..., 2., 1., 0.],
        [1., 1., 1.,  ..., 2., 2., 1.],
        ...,
        [0., 2., 2.,  ..., 1., 1., 0.],
        [1., 0., 2.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])

In [10]:
def model_0(sequences, lengths, emission_matrix, data_dim = 3, batch_size=None, include_prior=True, prediction=False):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(3) + 0.1)
                                  .to_event(1))

    tones_plate = pyro.plate("tones", data_dim, dim=-1)


    for i in pyro.plate("sequences", len(sequences)):
        length = lengths[i]
        sequence = sequences[i, :length]
        if prediction:
            print(f"{i} {length} {sequence}")
        x = 0
        for t in pyro.markov(range(length)):

            x = pyro.sample("x_{}_{}".format(i, t), dist.Categorical(probs_x[x]),
                            infer={"enumerate": "parallel"})
            with tones_plate:
                p_y_t = emission_matrix[x]

                pyro.sample("y_{}_{}".format(i, t), 
                            dist.Categorical(p_y_t),
                            obs=sequence[t])

In [11]:
num_observations = float(lengths.sum())
pyro.set_rng_seed(seed)
pyro.clear_param_store()


In [12]:

# We'll train using MAP Baum-Welch, i.e. MAP estimation while marginalizing
# out the hidden state x. This is accomplished via an automatic guide that
# learns point estimates of all of our conditional probability tables,
# named probs_*.
guide = AutoDelta(poutine.block(model_0, expose_fn=lambda msg: msg["name"].startswith("probs_")))

In [13]:
first_available_dim = -2


In [14]:
guide_trace = poutine.trace(guide).get_trace(
    sequences, lengths, emission)


In [15]:
model_trace = poutine.trace(
    poutine.replay(poutine.enum(model_0, first_available_dim), guide_trace)).get_trace(
    sequences, lengths, emission)
logging.info(model_trace.format_shapes())
print(model_trace.format_shapes())


 Trace Shapes:             
  Param Sites:             
 Sample Sites:             
  probs_x dist        | 3 3
         value        | 3 3
    tones dist        |    
         value      3 |    
sequences dist        |    
         value     51 |    
    x_0_0 dist        |    
         value   3  1 |    
    y_0_0 dist   3  3 |    
         value        |    
    x_0_1 dist   3  1 |    
         value 3 1  1 |    
    y_0_1 dist 3 1  3 |    
         value        |    
    x_0_2 dist 3 1  1 |    
         value   3  1 |    
    y_0_2 dist   3  3 |    
         value        |    
    x_0_3 dist   3  1 |    
         value 3 1  1 |    
    y_0_3 dist 3 1  3 |    
         value        |    
    x_0_4 dist 3 1  1 |    
         value   3  1 |    
    y_0_4 dist   3  3 |    
         value        |    
    x_0_5 dist   3  1 |    
         value 3 1  1 |    
    y_0_5 dist 3 1  3 |    
         value        |    
    x_0_6 dist 3 1  1 |    
         value   3  1 |    
    y_0_6 dist   3  

In [16]:
optim = Adam({'lr': learning_rate})


In [17]:
Elbo = TraceEnum_ELBO
elbo = Elbo(max_plate_nesting= 2,
            strict_enumeration_warning=True,
           )
svi = SVI(model_0, guide, optim, elbo)


In [18]:

# # We'll train on small minibatches.
for step in range(num_steps):
    print(f"\tTraining step: {step}")
    loss = svi.step(sequences, lengths, emission,)
    print('\t{: >5d}\t{}'.format(step, loss / num_observations))
    median = guide.median()["probs_x"].flatten()
    print("\t{}".format(median))

	Training step: 0
	    0	2.4954346660539217
	tensor([0.3559, 0.3220, 0.3220, 0.3220, 0.3559, 0.3220, 0.3115, 0.3443, 0.3443])
	Training step: 1
	    1	2.485734911151961
	tensor([0.3788, 0.3110, 0.3102, 0.3105, 0.3791, 0.3105, 0.2907, 0.3546, 0.3548])
	Training step: 2
	    2	2.4773988970588237
	tensor([0.4010, 0.3014, 0.2976, 0.2987, 0.4026, 0.2987, 0.2711, 0.3640, 0.3649])
	Training step: 3
	    3	2.4704718137254904
	tensor([0.4216, 0.2945, 0.2838, 0.2869, 0.4262, 0.2869, 0.2531, 0.3722, 0.3747])
	Training step: 4
	    4	2.4649211090686274
	tensor([0.4400, 0.2908, 0.2692, 0.2753, 0.4495, 0.2752, 0.2368, 0.3786, 0.3846])
	Training step: 5
	    5	2.4606634880514706
	tensor([0.4563, 0.2893, 0.2544, 0.2640, 0.4722, 0.2638, 0.2225, 0.3828, 0.3947])
	Training step: 6
	    6	2.457596124387255
	tensor([0.4708, 0.2893, 0.2400, 0.2534, 0.4937, 0.2530, 0.2102, 0.3844, 0.4053])
	Training step: 7
	    7	2.455581533394608
	tensor([0.4836, 0.2901, 0.2263, 0.2436, 0.5135, 0.2429, 0.2000, 0.3833, 0.41

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\cwild\.conda\envs\nic\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-ad9a27da10cf>", line 4, in <module>
    loss = svi.step(sequences, lengths, emission,)
  File "C:\Users\cwild\.conda\envs\nic\lib\site-packages\pyro\infer\svi.py", line 128, in step
    loss = self.loss_and_grads(self.model, self.guide, *args, **kwargs)
  File "C:\Users\cwild\.conda\envs\nic\lib\site-packages\pyro\infer\traceenum_elbo.py", line 405, in loss_and_grads
    elbo_particle = _compute_dice_elbo(model_trace, guide_trace)
  File "C:\Users\cwild\.conda\envs\nic\lib\site-packages\pyro\infer\traceenum_elbo.py", line 170, in _compute_dice_elbo
    log_factors = contract_tensor_tree(log_factors, sum_dims, ring=ring)
  File "C:\Users\cwild\.conda\envs\nic\lib\site-packages\pyro\ops\contract.py", line 189, in contract_tensor_tree
    ordinal, term = _contrac

TypeError: object of type 'NoneType' has no len()

In [19]:
from hmmlearn import hmm

In [27]:
model = hmm.MultinomialHMM(n_components=3, n_iter=10000, params="st", init_params="st")

In [30]:
model.emissionprob_ = emission.cpu().numpy().astype(float)

In [31]:
model.fit(sequences.cpu().numpy().reshape(-1, 1).astype(int), lengths.cpu().numpy().astype(int))

MultinomialHMM(init_params='st', n_components=3, n_iter=10000, params='st',
               random_state=RandomState(MT19937) at 0x175F7569048)

In [32]:
model.transmat_

array([[0.92999261, 0.03601261, 0.03399478],
       [0.03824865, 0.92332934, 0.03842201],
       [0.04142672, 0.03791173, 0.92066155]])

In [ ]:
np.zeros

In [ ]:
guide.median()["probs_x"]

In [40]:
def model_1(sequences, lengths, emission_matrix, data_dim = 3, batch_size=None, include_prior=True, prediction=False):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(3) + 0.1)
                                  .to_event(1))

    tones_plate = pyro.plate("tones", data_dim, dim=-1)


    with pyro.plate("sequences", len(sequences)):
        length = lengths[i]
        sequence = sequences[i, :length]
        if prediction:
            print(f"{i} {length} {sequence}")
        x = 0
        for t in pyro.markov(range(length)):

            x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
                            infer={"enumerate": "parallel"})
            with tones_plate:
                p_y_t = emission_matrix[x]

                pyro.sample("y_{}".format(t), 
                            dist.Categorical(p_y_t),
                            obs=sequence[t])

Warmup:   0%|                                                                                    | 0/600 [01:04, ?it/s]


In [41]:
nuts_kernel = pyro.infer.NUTS(model_1)
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=500, warmup_steps=100)
mcmc.run(sequences, lengths, emission, batch_size=batch_size)

Warmup:   0%|                                                                                    | 0/600 [00:00, ?it/s]

NameError: name 'i' is not defined

In [ ]:
mcmc.summary()

In [ ]:
def emission(x):
    return None

In [ ]:
def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        
    data_dim = 7
    
#     print(sequences)
#     print(lengths)
#     print(num_sequences)
#     print(max_length)


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
                                  .to_event(1))
        
    with pyro.plate("sequences", num_sequences, batch_size, dim=-2) as batch:
        lengths = lengths[batch]
        
        x = (6* torch.ones(batch_size)).to(torch.int)
        y = (6 * torch.ones(batch_size)).to(torch.int)
        
        for t in pyro.markov(range(max_length if args.jit else lengths.max())):
            with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
#                 print(f"prob_x: {x} {x} {probs_x[x]}")
                x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
                                infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x} {x} {probs_x[x].shape}")
                
                # Note that since each tone depends on all tones at a previous time step
                # the tones at different time steps now need to live in separate plates.
                with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
                    p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
                    y = pyro.sample("y_{}".format(t),
                                    dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
                                    obs=sequences[batch, t],
                                   )

In [ ]:
# def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True, prediction=False):
#     with ignore_jit_warnings():
#         num_sequences, max_length = map(int, sequences.shape)
#         assert lengths.shape == (num_sequences,)
#         assert lengths.max() <= max_length
        
#     data_dim = 7
    
# #     print(sequences)
# #     print(lengths)
# #     print(num_sequences)
# #     print(max_length)


#     with poutine.mask(mask=include_prior):
#         probs_x = pyro.sample("probs_x",
#                               dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
#                                   .to_event(1))

#     tones_plate = pyro.plate("tones", data_dim, dim=-1)


#     for i in pyro.plate("sequences", len(sequences), batch_size):
#         length = lengths[i]
#         sequence = sequences[i, :length]
#         if prediction:
#             print(f"{i} {length} {sequence}")
#         x = 6
#         for t in pyro.markov(range(length)):
#             # On the next line, we'll overwrite the value of x with an updated
#             # value. If we wanted to record all x values, we could instead
#             # write x[t] = pyro.sample(...x[t-1]...).
#             x = pyro.sample("x_{}_{}".format(i, t), dist.Categorical(probs_x[x]),
#                             infer={"enumerate": "parallel"})
#             with tones_plate:
#                 p_y_t = emission_matrix[x]

#                 pyro.sample("y_{}_{}".format(i, t), 
#                             dist.Categorical(p_y_t),
#                             obs=sequence[t])
        
#         for t in pyro.markov(range(max_length if args.jit else lengths.max())):
#             with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
# #                 print(f"prob_x: {x}")
#                 x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
#                                 infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x.shape}")
                
#                 # Note that since each tone depends on all tones at a previous time step
#                 # the tones at different time steps now need to live in separate plates.
#                 with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
#                     p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
#                     y = pyro.sample("y_{}".format(t),
#                                     dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
#                                     obs=sequences[batch, t],
#                                    )

In [ ]:
def model_0(sequences, lengths, emission_matrix, args, batch_size=None, include_prior=True, prediction=False):
    with ignore_jit_warnings():
        num_sequences, max_length = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
        
    data_dim = 7
    
#     print(sequences)
#     print(lengths)
#     print(num_sequences)
#     print(max_length)


    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample("probs_x",
                              dist.Dirichlet(0.9 * torch.eye(7) + 0.1)
                                  .to_event(1))

    tones_plate = pyro.plate("tones", data_dim, dim=-1)


    for i in pyro.plate("sequences", len(sequences)):
        length = lengths[i]
        sequence = sequences[i, :length]
        if prediction:
            print(f"{i} {length} {sequence}")
        x = 6
        for t in pyro.markov(range(length)):
            # On the next line, we'll overwrite the value of x with an updated
            # value. If we wanted to record all x values, we could instead
            # write x[t] = pyro.sample(...x[t-1]...).
            x = pyro.sample("x_{}_{}".format(i, t), dist.Categorical(probs_x[x]),
                            infer={"enumerate": "parallel"})
            with tones_plate:
                p_y_t = emission_matrix[x]

                pyro.sample("y_{}_{}".format(i, t), 
                            dist.Categorical(p_y_t),
                            obs=sequence[t])
        
#         for t in pyro.markov(range(max_length if args.jit else lengths.max())):
#             with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
# #                 print(f"prob_x: {x}")
#                 x = pyro.sample("x_{}".format(t), dist.Categorical(probs_x[x]),
#                                 infer={"enumerate": "parallel"})
#                 print(f"prob_x_sampled: {x.shape}")
                
#                 # Note that since each tone depends on all tones at a previous time step
#                 # the tones at different time steps now need to live in separate plates.
#                 with pyro.plate("tones_{}".format(t), data_dim, dim=-1):
#                     p_y_t = emission_matrix[x]
#                     print(f"p_y_t: {p_y_t.shape}")
#                     y = pyro.sample("y_{}".format(t),
#                                     dist.Categorical(p_y_t).expand([8, 1]),#.to_event(1),
#                                     obs=sequences[batch, t],
#                                    )

In [ ]:
# Utility function to print latent sites' quantile information.
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

In [ ]:
def main(args):
    if args.cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')

    logging.info('Loading data')
    data = load_sequences(args.dataset_path)

    logging.info('-' * 40)
    model = model_0
    logging.info('Training {} on {} sequences'.format(
        model.__name__, len(data['train']['sequences'])))
    
    sequences = data['train']['sequences']
#     print(sequences)
    lengths = data['train']['sequence_lengths']
#     print(lengths)
    
    emission_matrix_np = np.load(args.emission_matrix_path)
    emission_matrix = torch.Tensor(emission_matrix_np)
    # find all the notes that are present at least once in the training set
#     present_notes = ((sequences == 1).sum(0).sum(0) > 0)
    # remove notes that are never played (we remove 37/88 notes)
#     sequences = sequences[..., present_notes]

    if args.truncate:
        lengths = lengths.clamp(max=args.truncate)
        sequences = sequences[:, :args.truncate]
    num_observations = float(lengths.sum())
    pyro.set_rng_seed(args.seed)
    pyro.clear_param_store()

    # We'll train using MAP Baum-Welch, i.e. MAP estimation while marginalizing
    # out the hidden state x. This is accomplished via an automatic guide that
    # learns point estimates of all of our conditional probability tables,
    # named probs_*.
    guide = AutoDelta(poutine.block(model, expose_fn=lambda msg: msg["name"].startswith("probs_")))

    # To help debug our tensor shapes, let's print the shape of each site's
    # distribution, value, and log_prob tensor. Note this information is
    # automatically printed on most errors inside SVI.
#     if args.print_shapes:
    first_available_dim = -2 if model is model_0 else -3
    guide_trace = poutine.trace(guide).get_trace(
        sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
    model_trace = poutine.trace(
        poutine.replay(poutine.enum(model, first_available_dim), guide_trace)).get_trace(
        sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
    logging.info(model_trace.format_shapes())
    print(model_trace.format_shapes())

    # Enumeration requires a TraceEnum elbo and declaring the max_plate_nesting.
    # All of our models have two plates: "data" and "tones".
    optim = Adam({'lr': args.learning_rate})
    if args.tmc:
        if args.jit:
            raise NotImplementedError("jit support not yet added for TraceTMC_ELBO")
        elbo = TraceTMC_ELBO(max_plate_nesting=1 if model is model_0 else 2)
        tmc_model = poutine.infer_config(
            model,
            lambda msg: {"num_samples": args.tmc_num_samples, "expand": False} if msg["infer"].get("enumerate", None) == "parallel" else {})  # noqa: E501
        svi = SVI(tmc_model, guide, optim, elbo)
    else:
        Elbo = JitTraceEnum_ELBO if args.jit else TraceEnum_ELBO
        elbo = Elbo(max_plate_nesting= 2,
                    strict_enumeration_warning=True,
                    jit_options={"time_compilation": args.time_compilation})
        svi = SVI(model, guide, optim, elbo)

    # We'll train on small minibatches.
    logging.info('Step\tLoss')
    for step in range(args.num_steps):
        print(f"\tTraining step: {step}")
        loss = svi.step(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size)
        logging.info('{: >5d}\t{}'.format(step, loss / num_observations))
        print('\t{: >5d}\t{}'.format(step, loss / num_observations))
        
    return svi, model, guide, sequences, lengths, emission_matrix

    from pyro.infer import Predictive


    num_samples = 1000
    predictive = Predictive(model, guide=guide, num_samples=num_samples)
    svi_samples = {k: v.detach().cpu().numpy()
               for k, v in predictive(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size, prediction=True).items()
               if k != "obs"}
    
    print(svi_samples)
    
    for site, values in summary(svi_samples).items():
        print("Site: {}".format(site))
        print(values, "\n")
        

        
    print(pyro.param("probs_x").item())

    if args.jit and args.time_compilation:
        logging.debug('time to compile: {} s.'.format(elbo._differentiable_loss.compile_time))

    # We evaluate on the entire training dataset,
    # excluding the prior term so our results are comparable across models.
    train_loss = elbo.loss(model, guide, sequences, lengths, emission_matrix, args, include_prior=False)
    logging.info('training loss = {}'.format(train_loss / num_observations))
    print('training loss = {}'.format(train_loss / num_observations))

    # We expect models with higher capacity to perform better,
    # but eventually overfit to the training set.
    capacity = sum(value.reshape(-1).size(0)
                   for value in pyro.get_param_store().values())
    logging.info('{} capacity = {} parameters'.format(model.__name__, capacity))
    print('{} capacity = {} parameters'.format(model.__name__, capacity))


In [ ]:
# Content

In [ ]:

assert pyro.__version__.startswith('1.6.0')
parser = argparse.ArgumentParser(description="MAP Baum-Welch learning Bach Chorales")

parser.add_argument("-n", "--num-steps", default=100, type=int)
parser.add_argument("-b", "--batch-size", default=1, type=int)
parser.add_argument("-d", "--hidden-dim", default=16, type=int)
parser.add_argument("-nn", "--nn-dim", default=48, type=int)
parser.add_argument("-nc", "--nn-channels", default=2, type=int)
parser.add_argument("-lr", "--learning-rate", default=0.05, type=float)
parser.add_argument("-t", "--truncate", type=int)
parser.add_argument("-p", "--print-shapes", action="store_true")
parser.add_argument("--seed", default=0, type=int)
parser.add_argument('--cuda', action='store_true')
parser.add_argument('--jit', action='store_true')
parser.add_argument('--time-compilation', action='store_true')
parser.add_argument('-rp', '--raftery-parameterization', action='store_true')
parser.add_argument('--tmc', action='store_true',
                    help="Use Tensor Monte Carlo instead of exact enumeration "
                         "to estimate the marginal likelihood. You probably don't want to do this, "
                         "except to see that TMC makes Monte Carlo gradient estimation feasible "
                         "even with very large numbers of non-reparametrized variables.")
parser.add_argument('--tmc-num-samples', default=10, type=int)

parser.add_argument("--dataset_path")
parser.add_argument("--emission_matrix_path")



In [ ]:
args = parser.parse_args(["--dataset_path", f"{str(dataset_path)}", "--emission_matrix_path", f"{str(emission_matrix_path)}"])


In [ ]:
args

In [ ]:
svi, model, guide, sequences, lengths, emission_matrix = main(args)

In [ ]:
from pyro.infer import Predictive


num_samples = 1000
predictive = Predictive(model, guide=guide, num_samples=num_samples)


In [ ]:
svi_samples = {k: v.detach().cpu().numpy()
           for k, v in predictive(sequences, lengths, emission_matrix, args=args, batch_size=args.batch_size, include_prior=False).items()
           if k != "obs"}

In [ ]:
dir(svi)

In [ ]:
svi_samples["probs_x"].shape

In [ ]:
translation_matrix = np.mean(svi_samples["probs_x"], axis=0).reshape((7,7))

In [ ]:
pyro.get_param_store()

In [ ]:
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}

In [ ]:
confusion_matrix_test_table = pd.DataFrame(
data=translation_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
test_class_count = confusion_matrix_test_table.sum(axis=1)
test_class_count

In [ ]:
sequences[0,:]

In [ ]:
translation_matrix = np.mean(svi_samples["probs_x"], axis=0).reshape((7,7))

In [ ]:
confusion_matrix_test_table = pd.DataFrame(
data=translation_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
# Dataset loading
def load_sequences_np(dataset_path):
    sequences = {"train": {}}
    
#     sequence_list = [torch.Tensor(x) for x in np.load(str(dataset_path))]
    
    sequences_tensor = np.load(str(dataset_path))
    
    
    sequences["train"]["sequence_lengths"] = np.array([len(sequence) for sequence in sequences_tensor])
    sequences["train"]["sequences"] = sequences_tensor
    return sequences

In [ ]:
data = load_sequences_np(args.dataset_path)

In [ ]:
sequences = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [ ]:
sequences.reshape(-1)

In [ ]:
lengths

In [ ]:
emission_matrix_np = np.load(args.emission_matrix_path)


In [ ]:
from hmmlearn import hmm

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=10000, params="st", init_params="st")

In [ ]:
model.emissionprob_ = emission_matrix_np

In [ ]:
model.fit(sequences.reshape(-1, 1), lengths)

In [ ]:
model.emissionprob_

In [ ]:
model.transmat_

In [ ]:
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=50, params="st", init_params="st")
model.emissionprob_ = emission_matrix_np
model.fit(sequences.reshape(-1, 1), lengths)
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
model = hmm.MultinomialHMM(n_components=7, n_iter=500, params="st", init_params="st")
model.emissionprob_ = emission_matrix_np
model.fit(sequences.reshape(-1, 1), lengths)
classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
confusion_matrix_test_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table

In [ ]:
transition_matrix = np.array([[9.17269248e-001, 6.53003806e-118, 8.23362715e-002,
        7.72427778e-085, 1.53698963e-014, 3.43175139e-011,
        3.94480110e-004],
       [1.49172147e-161, 8.73311971e-001, 1.82502958e-049,
        1.26687948e-001, 8.14069900e-008, 1.46260187e-014,
        1.51752796e-026],
       [1.41727846e-003, 1.18840441e-019, 9.49950247e-001,
        1.82914991e-003, 9.07768488e-003, 2.60561958e-002,
        1.16694438e-002],
       [6.51786889e-084, 1.39464295e-002, 1.01244429e-018,
        8.97187712e-001, 8.36640492e-002, 4.86750640e-005,
        5.15313390e-003],
       [6.15269241e-003, 2.86600087e-010, 3.19099140e-002,
        4.01633777e-002, 9.05916897e-001, 1.58571186e-002,
        1.00840900e-032],
       [3.72329494e-005, 5.49847908e-003, 2.38550441e-002,
        3.82597321e-003, 7.01993013e-002, 8.82466367e-001,
        1.41176022e-002],
       [3.63508204e-009, 4.16695519e-002, 1.42071041e-001,
        2.19540222e-032, 6.30280002e-029, 1.49592737e-001,
        6.66666667e-001]])
print(f"HMM transitions")
print(f"Oriented to slanted: {transition_matrix[1, 4] + transition_matrix[3,4]}")
print(f"slanted to oriented: {transition_matrix[4, 1] + transition_matrix[4,3]}")
print(f"unoriented to slanted: {transition_matrix[0, 4] + transition_matrix[2,4]}")
print(f"slanted to unoriented: {transition_matrix[4, 0] + transition_matrix[4,2]}")


In [ ]:
sequences[1, :]

In [ ]:
model.decode(sequences[1,:].reshape(-1,1))

In [ ]:
sequences[0, :]

In [ ]:
model.decode(sequences[0,:].reshape(-1,1))

In [ ]:
naive_transition_matrix = np.zeros((7,7))
for sequence in sequences:
    prev_state = 6
    for state in sequence:
#         print(state)
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state

In [ ]:

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5, "No_sample": 6}
naive_transition_matrix = naive_transition_matrix / np.sum(naive_transition_matrix, axis=1).reshape((-1, 1))
print(naive_transition_matrix.sum(axis=1))

confusion_matrix_test_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
confusion_matrix_test_table 

In [ ]:
transition_matrix = naive_transition_matrix
print(f"Naive CNN transitions")
print(f"Oriented to slanted: {transition_matrix[1, 4] + transition_matrix[3,4]}")
print(f"slanted to oriented: {transition_matrix[4, 1] + transition_matrix[4,3]}")
print(f"unoriented to slanted: {transition_matrix[0, 4] + transition_matrix[2,4]}")
print(f"slanted to unoriented: {transition_matrix[4, 0] + transition_matrix[4,2]}")


In [ ]:
X, Z = model.sample(100)

In [ ]:
X

In [ ]:
Z